# Cardiomyocyte Content Prediction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

## Introduction

#### Stem cell-derived cardyomyocytes (CM) have great potential for ischemic heart disease treatment. For their in-vitro production, hPSC (human pluripotent stem cells) must differentiate into hPSC-CM. To gain insight into the CM production process, process parameters and measurements will be examined to determined those that are most influential on increasing CM production (*source paper)

### Main questions:
#### Which factors are most influential in increasing CM production?
#### Can CM production be accurately predicted with these factors?

In [9]:
# Importing data
train_df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test_data.csv')
# Ensuring both datasets have same number of features
assert len(train_df.columns) == len(test_df.columns)
# Combining both datasets
data = pd.concat([train_df, test_df], ignore_index=True)

#### New train and test sets will be made later to ensure a random split

## Data Cleaning

In [12]:
data.isna().sum().sum()

0

In [13]:
data.describe()

,dd0 Cell Density,dd0-dd1 Cell Density Gradient,dd1 Cell Density,dd1-dd2 Cell Density Gradient,dd2 Cell Density,dd2-dd3 Cell Density Gradient,dd3 Cell Density,dd3-dd5 Cell Density Gradient,dd5 Cell Density,dd5-dd7 Cell Density Gradient,...,dd1 Lactate Concentration,dd3 Lactate Concentration,dd5 Lactate Concentration,dd7 Lactate Concentration,dd0 Glucose Concentration,dd1 Glucose Concentration,dd3 Glucose Concentration,dd5 Glucose Concentration,dd7 Glucose Concentration,dd10 CM Content
count,60.000000,60.000000,60.000000,60.000000,60.00000,60.000000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.726333,0.306942,0.851463,0.759819,1.44745,0.461963,1.903242,-0.062192,1.695008,-0.030869,...,13.263147,14.983333,14.678624,13.037513,9.814893,4.170149,1.607243,2.760942,2.849942,66.172667
std,0.250183,0.586726,0.235372,0.675301,0.47042,0.743174,0.586961,0.338154,0.606149,0.331125,...,2.431586,3.792930,3.327453,3.320086,1.007209,1.460220,2.528723,1.875777,2.192971,29.385584
min,0.295000,-0.755274,0.290000,-0.392857,0.34000,-0.462059,0.685000,-0.781250,0.490000,-0.794344,...,8.800000,0.995000,5.900000,5.900000,7.135000,0.600000,0.000000,0.000000,0.000000,5.360000
25%,0.555000,-0.039279,0.697125,0.200316,1.15525,0.067512,1.395000,-0.290455,1.212500,-0.239271,...,11.500000,14.616667,12.182500,11.090000,9.162500,3.370000,0.000000,1.351250,0.775000,47.675000
50%,0.670000,0.239709,0.840000,0.672321,1.38000,0.245569,2.022500,-0.091430,1.640000,-0.083716,...,13.200000,16.150000,14.752500,13.187500,9.520000,4.442500,0.900000,2.285000,2.885000,75.350000
75%,0.897500,0.521429,1.000000,1.051010,1.79250,0.593039,2.285000,0.131216,2.112500,0.162875,...,14.068750,16.572500,17.500000,15.391250,10.502500,5.300000,1.910347,4.102500,4.295000,92.000000
max,1.280000,2.625000,1.360000,3.567308,2.40000,4.128648,3.000000,0.810945,3.060000,0.793750,...,19.590000,18.730000,20.375000,20.590000,12.400000,6.800000,10.270000,8.100000,8.700000,97.400000
